In [77]:
from collections import defaultdict
from copy import deepcopy

f = open("input.txt", "r")

class Brick:
    def __init__(self, start, end) -> None:
        self.start = start
        self.end = end
        self.final_z_start = self.start[2]
        self.final_z_end = self.end[2]
        self.supporting_bricks = set()
        self.supported_bricks = set()

    def supportBrick(self, brick):
        self.supported_bricks.add(brick)

    def isSupportedBy(self, brick):
        self.supporting_bricks.add(brick)

    def removeSupportFrom(self, brick):
        if brick in self.supporting_bricks:
            self.supporting_bricks.remove(brick)

bricks_for_z = defaultdict(list)
z_with_bricks = set()
bricks = []

for line in f:
    [start_coords, end_coords] = line.replace('\n','').split('~')
    start_coords = [int(n) for n in start_coords.split(',')]
    end_coords = [int(n) for n in end_coords.split(',')]
    new_brick = Brick(start_coords, end_coords)
    starting_z = start_coords[2]
    bricks_for_z[starting_z].append(new_brick)
    bricks.append(new_brick)
    z_with_bricks.add(starting_z)

z_with_bricks = sorted(list(z_with_bricks))
height = z_with_bricks[-1]

grid_for_z = [{} for z in range(height+1)]

# For a give height, show all the coordinates where bricks are present
def printGridForZ(z):
    print(f"Grid for z {z}")
    for (x, y), brick in grid_for_z[z].items():
        print(f"({x}, {y}): {brick.start}~{brick.end}")

for z in z_with_bricks:
    # We process bricks by order of starting height
    while len(bricks_for_z[z]) > 0:
        # Process one brick at a time
        brick = bricks_for_z[z].pop()
        start = brick.start
        end = brick.end
        z_to_check = start[2]-1
        # Figure out the first height at which there is a potential obstacle
        while z_to_check > len(grid_for_z):
            z_to_check-=1
        # If we've reached the ground, there's no way to go down further
        can_go_down = (z_to_check >= 1)
        while can_go_down and z_to_check >= 1:
            if z_to_check < len(grid_for_z):
                # The grid represents the coordinates where a brick is present
                grid = grid_for_z[z_to_check]   
                x = start[0]
                while x <= end[0]:
                    y = start[1]
                    while y <= end[1]:
                        # For each block of the current brick, we check if its coordinates
                        # are already taken by another brick at this height
                        if (x,y) in grid:
                            # If they are, the brick stops here and we register the supporter/supported relation
                            can_go_down = False
                            grid[(x,y)].supportBrick(brick)
                            brick.isSupportedBy(grid[(x,y)])
                        y+=1
                    x+=1
            # We keep going down if the brick hasn't encountered an obstacle (other brick or ground)
            if can_go_down:
                z_to_check -= 1
        z_to_fill = z_to_check+1
        # We've found the final resting height of the brick, now we need to update the grid
        # for each height occupied by the brick
        for zz in range(z_to_fill, z_to_fill + end[2]-start[2]+1):
            for x in range(start[0], end[0]+1):
                for y in range(start[1], end[1]+1):
                    grid_for_z[zz][(x, y)] = brick
        # Finally we register the final resting place of the brick
        brick.final_z_start = z_to_fill
        brick.final_z_end = z_to_fill + end[2]-start[2]

# for z in range(len(grid_for_z)):
#     printGridForZ(z)

total_nb_of_bricks_that_can_be_disintegrated = 0

for brick in bricks:
    supported_bricks = [f"{b.start}~{b.end}" for b in brick.supported_bricks]
    supporting_bricks = [f"{b.start}~{b.end}" for b in brick.supporting_bricks]
    can_be_disintegrated = True
    for b in brick.supported_bricks:
        if len(b.supporting_bricks) == 1:
            # If one supported bricks has no other supporting bricks, it will fall
            can_be_disintegrated = False
            break
    if can_be_disintegrated:
        total_nb_of_bricks_that_can_be_disintegrated += 1

print(total_nb_of_bricks_that_can_be_disintegrated)

def disintegrateBrick(brick):
    for b in brick.supported_bricks:
        b.removeSupportFrom(brick)
        if len(b.supporting_bricks) == 0:
            # The brick b has no support any more, it's going to fall
            # So we need to simulate its fall next
            disintegrateBrick(b)

def countUnsupportedBricks(bricks):
    total = 0
    for brick in bricks:
        # If the brick is resting on the ground, it's supported
        if brick.final_z_start > 1 and len(brick.supporting_bricks) == 0:
            total += 1
    return total

def printStack(bricks):
    # Print all bricks in order
    for brick in bricks:
        supported_bricks = [f"{b.start}~{b.end}" for b in brick.supported_bricks]
        supporting_bricks = [f"{b.start}~{b.end}" for b in brick.supporting_bricks]
        print(f"{brick.start}~{brick.end} resting at z={brick.final_z_start},{brick.final_z_end} supporting {supported_bricks} and supported by {supporting_bricks}")

total = 0
brick_index = 0
while brick_index < len(bricks):
    # We're going to simulate the chain reaction for each brick, so we need to work on a copy
    bricks_copy = deepcopy(bricks)
    brick = bricks_copy[brick_index]
    supported_bricks = [f"{b.start}~{b.end}" for b in brick.supported_bricks]
    supporting_bricks = [f"{b.start}~{b.end}" for b in brick.supporting_bricks]
    print(f"Brick {brick_index} out of {len(bricks)}")
    print(f"Disintegrating {brick.start}~{brick.end} resting at z={brick.final_z_start},{brick.final_z_end} supporting {supported_bricks} and supported by {supporting_bricks}")
    disintegrateBrick(brick)
    falling_bricks = countUnsupportedBricks(bricks_copy)
    print(falling_bricks)
    total += falling_bricks
    brick_index+=1

print(total)

465
Brick 0 out of 1394
Disintegrating [5, 5, 156]~[5, 7, 156] resting at z=83,83 supporting ['[3, 6, 158]~[5, 6, 158]'] and supported by ['[5, 7, 154]~[7, 7, 154]']
521
Brick 1 out of 1394
Disintegrating [8, 5, 82]~[8, 7, 82] resting at z=41,41 supporting ['[8, 5, 83]~[9, 5, 83]'] and supported by ['[8, 4, 81]~[8, 6, 81]']
1
Brick 2 out of 1394
Disintegrating [6, 5, 196]~[6, 7, 196] resting at z=99,99 supporting ['[5, 6, 198]~[7, 6, 198]', '[6, 7, 198]~[6, 9, 198]', '[3, 5, 202]~[6, 5, 202]'] and supported by ['[5, 7, 193]~[9, 7, 193]']
4
Brick 3 out of 1394
Disintegrating [0, 7, 7]~[0, 8, 7] resting at z=3,3 supporting ['[0, 7, 8]~[0, 7, 10]'] and supported by ['[0, 8, 4]~[0, 9, 4]']
1
Brick 4 out of 1394
Disintegrating [2, 3, 129]~[2, 6, 129] resting at z=63,63 supporting ['[0, 6, 130]~[2, 6, 130]'] and supported by ['[1, 6, 125]~[3, 6, 125]', '[2, 4, 126]~[4, 4, 126]']
1
Brick 5 out of 1394
Disintegrating [9, 5, 265]~[9, 7, 265] resting at z=127,127 supporting ['[9, 4, 268]~[9, 6, 